# MENA Analysis: Air Quality NO2

### Load packages and setup

In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import rasterio
from rasterstats import zonal_stats

In [2]:
# Define directories
dir_shp = 'M:/MENA/GEO/Boundaries/'
dir_in = 'M:/MENA/GEO/Hazards/NO2/raw/'
dir_out = 'M:/MENA/GEO/Hazards/NO2/final/'


# Define output FILE
VAR = 'NO2_'
YEAR_LIST = [2019,2020]
MONTH_LIST = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12']
FILE_OUT = 'ADM2_' + VAR + str(min(YEAR_LIST)) + '_' + str(max(YEAR_LIST)) + '.csv'

In [4]:
%matplotlib inline

### Load shapefile

In [5]:
# Load shapefile
MENA_shp = gpd.read_file(dir_shp + 'MENA_ADM2.shp')

MENA_shp.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

### Load friction raster

In [9]:
df_full = pd.DataFrame(MENA_shp['ID_ADM'])

for YEAR in YEAR_LIST:
    for MONTH in MONTH_LIST:
        with rasterio.open(dir_in + 'NO2_' + str(YEAR)+ '_' + str(MONTH) + '.tif') as raster:
            array = raster.read(1)
            affine = raster.transform
            nodata = raster.nodata

        MENA_shp.to_crs(raster.crs, inplace = True)

        # Extract raster by polygon
        dict_temp = zonal_stats(MENA_shp, array, affine = affine, nodata = nodata, 
                                interpolate = 'bilinear', all_touched = True,
                                stats = ['mean'])
        df_temp = pd.DataFrame(dict_temp)
        df_temp = df_temp.add_prefix(VAR)

        df_temp = df_temp.add_suffix('_' + str(YEAR)+ str(MONTH))
        df_temp.insert(0, 'ID_ADM', MENA_shp['ID_ADM'])
    
        df_full = df_full.merge(df_temp, how = 'left', on = 'ID_ADM')

In [10]:
df_full

,ID_ADM,NO2_mean_20191,NO2_mean_20192,NO2_mean_20193,NO2_mean_20194,NO2_mean_20195,NO2_mean_20196,NO2_mean_20197,NO2_mean_20198,NO2_mean_20199,...,NO2_mean_20203,NO2_mean_20204,NO2_mean_20205,NO2_mean_20206,NO2_mean_20207,NO2_mean_20208,NO2_mean_20209,NO2_mean_202010,NO2_mean_202011,NO2_mean_202012
0,37631,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.000062,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,37632,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.000058,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,37633,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.000063,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,37634,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.000059,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,37635,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.000054,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2932,15561,0.000102,0.000094,0.000084,0.000088,0.000093,0.000092,0.000087,0.000082,0.000074,...,0.000081,0.000082,0.000089,0.000091,0.000081,0.000079,0.000072,0.000072,0.000080,0.000088
2933,15562,0.000053,0.000056,0.000062,0.000065,0.000069,0.000074,0.000070,0.000066,0.000057,...,0.000059,0.000061,0.000067,0.000067,0.000066,0.000065,0.000058,0.000053,0.000052,0.000046
2934,15563,0.000067,0.000066,0.000068,0.000077,0.000086,0.000085,0.000084,0.000076,0.000069,...,0.000068,0.000072,0.000080,0.000083,0.000080,0.000074,0.000069,0.000066,0.000060,0.000058
2935,15564,0.000072,0.000072,0.000070,0.000074,0.000077,0.000077,0.000076,0.000071,0.000062,...,0.000068,0.000066,0.000072,0.000071,0.000070,0.000070,0.000062,0.000058,0.000064,0.000061


In [11]:
df_full.to_csv(dir_out + 'NO2_2019_2020.csv', index = False)